In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i = PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [4]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i
1,4326,0.5758,4326,3.0471,2681,0.097191
2,4362,0.5781,2543,2.9807,2582,0.121122
3,4332,0.5862,4316,2.8843,1066,0.121128


In [5]:
#return the total power of node i.
def TPN(i):
    PI_ofi = Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    n1=TC_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n2=PI_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n3=CC_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    #equation []
    #TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    return TPN_i

In [6]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [7]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1], reverse=True)
TPN_Rank

[[2543, 34.09685541596704],
 [4326, 33.93056675363894],
 [4316, 32.08010015927552],
 [1066, 31.243146637037494],
 [2681, 29.19838120401393],
 [4362, 26.882944912026765],
 [2582, 24.669912581524542],
 [4332, 22.43010580744349]]

In [8]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:2])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(TPN_table)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

   nodes_TPN      TPN_i
0       2543  34.096855
1       4326  33.930567
2       4316  32.080100
3       1066  31.243147
4       2681  29.198381
5       4362  26.882945
6       2582  24.669913
7       4332  22.430106


In [9]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_FiveR = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_FiveR.index = Top_FiveR.index + 1
#convert column nodes_TPN to int
Top_FiveR.nodes_TPN= Top_FiveR.nodes_TPN.astype(int)
ranking_table = pd.concat([Top_FiveR['nodes_TC'], Top_FiveR['nodes_PI'], Top_FiveR['nodes_CC'],Top_FiveR['nodes_TPN']], axis=1, sort=False)
print(ranking_table)
#Propose_Table.to_csv('result/Propose_Table_power_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC  nodes_PI  nodes_CC  nodes_TPN
1      4326      4326      2681       2543
2      4362      2543      2582       4326
3      4332      4316      1066       4316


In [19]:
# calculate the kendall's correlation between two variables
from numpy.random import rand
from numpy.random import seed
from scipy.stats import kendalltau

# prepare data
data1 = Top_FiveR['nodes_TPN']
data2 = data1 + Top_FiveR['nodes_CC']+ Top_FiveR['nodes_PI']+ Top_FiveR['nodes_TC']
# calculate kendall's correlation
coef, p = kendalltau(data1, data2)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)


Kendall correlation coefficient: -0.333
Samples are uncorrelated (fail to reject H0) p=0.602


In [11]:
# Importing required libraries
import researchpy as rp
from scipy import stats
#The casewise method returns all information in 3 dataframes
#corr_type, corr_matrix, corr_ps=>
corr_type, corr_matrix, corr_ps = rp.corr_case(Top_FiveR[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])
#PAIRWISEcorrelation
corr_pair = rp.corr_pair(Top_FiveR[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])